Streaming data dan data preprocessing Kelompok 5 PDB

12S18002 : Wiranda Siahaan

12S18009 : Novita Enjelia Hutapea

12S18041 : Merika H. Manurung

12S18046 : Tiara Octavia Situmorang

# Consume twitter steaming data

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
import socket
import json
import numpy as np
import pandas as pd

In [3]:
#Kindly put your credentials received from twitter developer account
consumer_key='2mTmxHwkZramjVDOewE2dybgQ'
consumer_secret='dJ42E93prTAze5BoF4wAIcxYA6gXfu9rGrHr6gguUt85xcHJSi'
access_token ='1517128047019040768-YMZCcFIf6VLbZXNkJsa79sZk70xFkM'
access_secret='Auwa9WPIktjUCrgQmzceZsdMtMA2hbnebSR7QlmYEdrlq'

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [5]:
for tweet in api.search(q='miniso', lan='en'):    
    print(tweet.text)

👦 : kak hair dryer harganya berapa?
💁 : Tergantung merek, 200k sampe jutaan juga ada
👦 : Aku pengen styling rambut… https://t.co/8AvYBbPdUS
RT @IanCorpel: Necesito ir a Miniso a comprar cosas que  o sabia que necesitaba
@convomfs joop, vs, miniso
@PP_miniso น่ารัก
Si quieren hacerme feliz regálenme cualquier cosa de Tom &amp; Jerry de Miniso, soy la más feliz 🤍🤍🤍
Pict 2
Bucket Hat : Miniso
Kemeja tartan : https://t.co/8TiXmKQS05
Inner kemeja : https://t.co/6qvbBrdAiq
Kulot jea… https://t.co/oi8uswCj4i
RT @JapanSalaryboom: พ่อแม่คนไหนกำลังประสบปัญหาลูกติดจอ ติดมือถือ อยากให้ลองเอาไปใช้กับลูกน้อยของคุณดูครับ
.
แนะนำโดย T-Sensei ผู้เชี่ยวชาญ…
Por $50 pesos confieso que tienen buena definición los audífonos de miniso.
• • • #pic #geek https://t.co/Xjll9aD5gV
@convomfs parfum miniso yg gambar nanas tuh apa ya namanya lupa
Me encontré a la @JenySweet1 en el Miniso. https://t.co/f8KSXfhqw3
3. Miniso Pouch

https://t.co/qWnRCpeAvp
💰25.900-49.900 https://t.co/aaVudLwQAe
@PP_miniso ใช่ครับใช่
@r

In [6]:
tweetDf = pd.DataFrame(columns = ['User', 'User_statuses_count', 
                             'user_followers','fav_count','User_location','Tweets',])

In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100000, lang='en').items():
        print(i, end='\r')
        tweetDf.loc[i, 'User'] = tweet.user.name
        tweetDf.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        tweetDf.loc[i, 'user_followers'] = tweet.user.followers_count
        tweetDf.loc[i, 'fav_count'] = tweet.favorite_count
        tweetDf.loc[i, 'User_location'] = tweet.user.location
        tweetDf.loc[i, 'Tweets'] = tweet.text
        
        
        tweetDf.to_csv('{}.csv'.format(file_name))
        i+=1
        if i == 200:
            break
        else:
            pass

In [8]:
stream(data = ['miniso'], file_name = 'miniso_2021')

In [9]:
tweetDf.head()

,User,User_statuses_count,user_followers,fav_count,User_location,Tweets
0,r❕beatbox 🗯️🫧 | ebbg pero may exceptions,1118,241,0,\n𝄒𓏲࣪ .she/her,"RT @lexhaul: wts lfb ph [help rt please, tysm!..."
1,h3lluh,8266,239,0,/ svt / atz / nct / en- /,RT @indyloopz: WE NEED 100M STREAMS FOR THE HO...
2,ven ⛓️,1013,498,0,she / her (minor) ✿ ✿ ✿,"RT @lexhaul: wts lfb ph [help rt please, tysm!..."
3,zay,17680,1051,0,txt member 🐿️,"@convomfs danish green, miniso"
4,h3lluh,8266,239,0,/ svt / atz / nct / en- /,bump https://t.co/waqunEFwxa


## Data Preprocessing

In [10]:
# libraries
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # used for encoding categorical data
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.preprocessing import StandardScaler # used for feature scaling

In [11]:
dataset = pd.read_csv('miniso_2021.csv') # to import the dataset into a variable
# Splitting the attributes into independent and dependent attributes
X = dataset.iloc[:, :-1].values # attributes to determine dependent variable / Class
Y = dataset.iloc[:, -1].values # dependent variable / Class

In [12]:
dataset.head()

,Unnamed: 0,User,User_statuses_count,user_followers,fav_count,User_location,Tweets
0,0,r❕beatbox 🗯️🫧 | ebbg pero may exceptions,1118,241,0,\n𝄒𓏲࣪ .she/her,"RT @lexhaul: wts lfb ph [help rt please, tysm!..."
1,1,h3lluh,8266,239,0,/ svt / atz / nct / en- /,RT @indyloopz: WE NEED 100M STREAMS FOR THE HO...
2,2,ven ⛓️,1013,498,0,she / her (minor) ✿ ✿ ✿,"RT @lexhaul: wts lfb ph [help rt please, tysm!..."
3,3,zay,17680,1051,0,txt member 🐿️,"@convomfs danish green, miniso"
4,4,h3lluh,8266,239,0,/ svt / atz / nct / en- /,bump https://t.co/waqunEFwxa


In [13]:
print(dataset.shape)

(200, 7)


### drop coloumn

In [14]:
to_drop = ['Unnamed: 0', 'User', 'User_statuses_count', 'user_followers', 'fav_count', 'User_location']

In [15]:
dataset.drop(to_drop, inplace=True, axis = 1)

In [16]:
dataset.head()

,Tweets
0,"RT @lexhaul: wts lfb ph [help rt please, tysm!..."
1,RT @indyloopz: WE NEED 100M STREAMS FOR THE HO...
2,"RT @lexhaul: wts lfb ph [help rt please, tysm!..."
3,"@convomfs danish green, miniso"
4,bump https://t.co/waqunEFwxa


In [17]:
print(dataset.shape)

(200, 1)


### missing value

In [18]:
# handling the missing data and replace missing values with nan from numpy and replace with mean of all the other values
dataset1 = dataset.dropna()
print(dataset1.shape)

(200, 1)


# Text Preprocessing



In [19]:
import re, string, unicodedata
import json
import nltk
import json
import time
import itertools  
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS
import tensorflow.keras.preprocessing.text as kpt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

## Case folding

In [20]:
# Change text column to lower lower case
dataset1['Tweets'] = dataset1['Tweets'].str.lower()

# Look at the change
print(dataset1)

                                                Tweets
0    rt @lexhaul: wts lfb ph [help rt please, tysm!...
1    rt @indyloopz: we need 100m streams for the ho...
2    rt @lexhaul: wts lfb ph [help rt please, tysm!...
3                       @convomfs danish green, miniso
4                         bump https://t.co/waqunefwxa
..                                                 ...
195  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
196                  they have gay pins at miniso slay
197  rt @sugarsanriio: hello kitty cooler bag from ...
198  rt @sugarsanriio: hello kitty cooler bag from ...
199  @wefinallyhere online i don't know, miniso is ...

[200 rows x 1 columns]


## Remove URL

In [21]:
def clean_data(dataset2):
#replace URL of a text
    dataset1['Tweets'] = dataset2['Tweets'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

clean_data(dataset1)
print(dataset1['Tweets']);

0      rt @lexhaul: wts lfb ph [help rt please, tysm!...
1      rt @indyloopz: we need 100m streams for the ho...
2      rt @lexhaul: wts lfb ph [help rt please, tysm!...
3                         @convomfs danish green, miniso
4                                                 bump  
                             ...                        
195    rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
196                    they have gay pins at miniso slay
197    rt @sugarsanriio: hello kitty cooler bag from ...
198    rt @sugarsanriio: hello kitty cooler bag from ...
199    @wefinallyhere online i don't know, miniso is ...
Name: Tweets, Length: 200, dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


## Remove Number

In [22]:
# removing numbers from strings of speciafied 
# column, here 'Name'
dataset1['Tweets']= dataset1['Tweets'].str.replace('\d+', '')
  
# display output with numbers removed from 
# required strings
print(dataset1)

                                                Tweets
0    rt @lexhaul: wts lfb ph [help rt please, tysm!...
1    rt @indyloopz: we need m streams for the hot m...
2    rt @lexhaul: wts lfb ph [help rt please, tysm!...
3                       @convomfs danish green, miniso
4                                               bump  
..                                                 ...
195  rt @kyvo_gvr: wts | lfb [help rt] \n\n𓂋 xiumin...
196                  they have gay pins at miniso slay
197  rt @sugarsanriio: hello kitty cooler bag from ...
198  rt @sugarsanriio: hello kitty cooler bag from ...
199  @wefinallyhere online i don't know, miniso is ...

[200 rows x 1 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


## Remove Punctuation

In [23]:
dataset1['Tweets'] = dataset1['Tweets'].str.replace(r'[^\w\s]','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [24]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'\s*\d+\s*', '', text)
    text = re.sub('/[^\w\s]/gi', '', text)
    text = re.sub("@[^0-9a-zA-Z]+", '',text)
    text = re.sub(r'@[^\s]+', ' ', text)
    text = re.sub(r'#[^\w]+', ' ', text)
    text = re.sub(r"[.,:;+!\-_<^/=?&|$\"\(\)\d\*]", " ", text)
    text = re.sub('  +', ' ', text) 
    text = re.sub('\n+',' ',text)
    text = re.sub('\n', text)
    text = re.sub('\ ■',' ',text)
    text = re.sub(r'&[^\s]+', ' ', text)


    return text

In [25]:
clean_data(dataset1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
print(dataset1)

                                                Tweets
0    rt lexhaul wts lfb ph help rt please tysm\noff...
1    rt indyloopz we need m streams for the hot mv ...
2    rt lexhaul wts lfb ph help rt please tysm\noff...
3                         convomfs danish green miniso
4                                               bump  
..                                                 ...
195  rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
196                  they have gay pins at miniso slay
197  rt sugarsanriio hello kitty cooler bag from mi...
198  rt sugarsanriio hello kitty cooler bag from mi...
199  wefinallyhere online i dont know miniso is the...

[200 rows x 1 columns]


## Stopwords Removal

In [27]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [28]:
print(dataset1['Tweets']); 

import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
 
dataset1['Tweets'] = dataset1['Tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
print(dataset1['Tweets'])

0      rt lexhaul wts lfb ph help rt please tysm\noff...
1      rt indyloopz we need m streams for the hot mv ...
2      rt lexhaul wts lfb ph help rt please tysm\noff...
3                           convomfs danish green miniso
4                                                 bump  
                             ...                        
195    rt kyvo_gvr wts  lfb help rt \n\n𓂋 xiumin exop...
196                    they have gay pins at miniso slay
197    rt sugarsanriio hello kitty cooler bag from mi...
198    rt sugarsanriio hello kitty cooler bag from mi...
199    wefinallyhere online i dont know miniso is the...
Name: Tweets, Length: 200, dtype: object
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0      rt lexhaul wts lfb ph help rt please tysm offi...
1      rt indyloopz need streams hot mv youre

## Tokenizing

In [29]:
dataset1['Tweets'] = dataset1['Tweets'].apply(word_tokenize)

print(dataset1['Tweets'])

0      [rt, lexhaul, wts, lfb, ph, help, rt, please, ...
1      [rt, indyloopz, need, streams, hot, mv, youre,...
2      [rt, lexhaul, wts, lfb, ph, help, rt, please, ...
3                      [convomfs, danish, green, miniso]
4                                                 [bump]
                             ...                        
195    [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
196                            [gay, pins, miniso, slay]
197    [rt, sugarsanriio, hello, kitty, cooler, bag, ...
198    [rt, sugarsanriio, hello, kitty, cooler, bag, ...
199    [wefinallyhere, online, dont, know, miniso, ri...
Name: Tweets, Length: 200, dtype: object


In [30]:
print(dataset1)

                                                Tweets
0    [rt, lexhaul, wts, lfb, ph, help, rt, please, ...
1    [rt, indyloopz, need, streams, hot, mv, youre,...
2    [rt, lexhaul, wts, lfb, ph, help, rt, please, ...
3                    [convomfs, danish, green, miniso]
4                                               [bump]
..                                                 ...
195  [rt, kyvo_gvr, wts, lfb, help, rt, 𓂋, xiumin, ...
196                          [gay, pins, miniso, slay]
197  [rt, sugarsanriio, hello, kitty, cooler, bag, ...
198  [rt, sugarsanriio, hello, kitty, cooler, bag, ...
199  [wefinallyhere, online, dont, know, miniso, ri...

[200 rows x 1 columns]


## Hasil Akhir

In [31]:
dataset1.to_csv('/content/gdrive/My Drive/pdb/preprocessing_PDB.csv', index = False)

In [32]:
data = pd.read_csv('/content/gdrive/My Drive/pdb/preprocessing_PDB.csv')

In [33]:
from textblob import TextBlob

def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity

def getPolarity(review):
    return TextBlob(review).sentiment.polarity

def analyze(score):
    if score  < 0:
        return '-1'
    elif score == 0:
        return '0'
    else:
        return '1'

In [34]:
final_data = pd.DataFrame(data[['Tweets']])
final_data.head()

,Tweets
0,"['rt', 'lexhaul', 'wts', 'lfb', 'ph', 'help', ..."
1,"['rt', 'indyloopz', 'need', 'streams', 'hot', ..."
2,"['rt', 'lexhaul', 'wts', 'lfb', 'ph', 'help', ..."
3,"['convomfs', 'danish', 'green', 'miniso']"
4,['bump']


In [35]:
final_data['Subjectivity'] = final_data['Tweets'].apply(getSubjectivity)
final_data['Polarity'] =  final_data['Tweets'].apply(getPolarity)
final_data['TextBlob'] = final_data['Polarity'].apply(analyze)
final_data.head()

,Tweets,Subjectivity,Polarity,TextBlob
0,"['rt', 'lexhaul', 'wts', 'lfb', 'ph', 'help', ...",0.10,0.050000,1
1,"['rt', 'indyloopz', 'need', 'streams', 'hot', ...",0.65,-0.116667,-1
2,"['rt', 'lexhaul', 'wts', 'lfb', 'ph', 'help', ...",0.10,0.050000,1
3,"['convomfs', 'danish', 'green', 'miniso']",0.30,-0.200000,-1
4,['bump'],0.00,0.000000,0


In [36]:
tb_counts = final_data.TextBlob.value_counts()
tb_counts

0     111
1      78
-1     11
Name: TextBlob, dtype: int64

In [37]:
final_data.to_csv('/content/gdrive/My Drive/pdb/Final_Data.csv', index = False)